In [6]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [7]:
books = pd.read_csv("../data/BX-Books.csv", sep=";", encoding="latin-1", on_bad_lines='warn')

C:\Users\USER\AppData\Local\Temp\ipykernel_7580\2964136533.py:1: ParserWarning: Skipping line 6452: expected 8 fields, saw 9
Skipping line 43667: expected 8 fields, saw 10
Skipping line 51751: expected 8 fields, saw 9

  books = pd.read_csv("../data/BX-Books.csv", sep=";", encoding="latin-1", on_bad_lines='warn')
C:\Users\USER\AppData\Local\Temp\ipykernel_7580\2964136533.py:1: ParserWarning: Skipping line 92038: expected 8 fields, saw 9
Skipping line 104319: expected 8 fields, saw 9
Skipping line 121768: expected 8 fields, saw 9

  books = pd.read_csv("../data/BX-Books.csv", sep=";", encoding="latin-1", on_bad_lines='warn')
C:\Users\USER\AppData\Local\Temp\ipykernel_7580\2964136533.py:1: ParserWarning: Skipping line 144058: expected 8 fields, saw 9
Skipping line 150789: expected 8 fields, saw 9
Skipping line 157128: expected 8 fields, saw 9
Skipping line 180189: expected 8 fields, saw 9
Skipping line 185738: expected 8 fields, saw 9

  books = pd.read_csv("../data/BX-Books.csv", sep=";

In [8]:
books.head(2)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...


In [9]:
books = books[["ISBN", "Book-Title", "Book-Author", "Year-Of-Publication", "Publisher", "Image-URL-L"]]

In [10]:
books.head(2)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...


In [11]:
books.rename(columns={
    "Book-Title": "title",
    "Book-Author": "author",
    "Year-Of-Publication": "publication_year",
    "Publisher": "publisher",
    "Image-URL-L": "image_url"
}, inplace=True)

In [12]:
books.head(2)

,ISBN,title,author,publication_year,publisher,image_url
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...


In [13]:
users = pd.read_csv("./data/BX-Users.csv", sep=";", encoding="latin-1", on_bad_lines='warn')

FileNotFoundError: [Errno 2] No such file or directory: './data/BX-Users.csv'

In [ ]:
users.head()

In [ ]:
rating = pd.read_csv("./data/BX-Book-Ratings.csv", sep=";", encoding='latin-1', on_bad_lines='warn')

FileNotFoundError: [Errno 2] No such file or directory: './data/BX-Book-Ratings.csv'

In [ ]:
rating.head()
# rating.shape

NameError: name 'rating' is not defined

In [ ]:
rating.rename(columns={
    "User-ID": "user_id",
    "Book-Rating": "rating",
}, inplace=True)

In [ ]:
print(books.shape)
print(users.shape)
print(rating.shape)

In [ ]:
rating['user_id'].value_counts()

## showing how many of the users give 200 + rating

In [ ]:
x = rating['user_id'].value_counts() > 200
# before
print("before: ",x.shape) # 10Lakh plus
#after
print("after: ",x[x].shape) # only 899

In [ ]:
#storing the users index who rate 200+

y = x[x].index

In [ ]:
ratings = rating[rating['user_id'].isin(y)]

In [ ]:
ratings.head()

In [ ]:
rating_with_books = ratings.merge(books, on="ISBN")

In [ ]:
rating_with_books.head(2)
# rating_with_books.shape

In [ ]:
books_num_rating = rating_with_books.groupby('title')['rating'].count().reset_index()

In [ ]:
books_num_rating.head()

In [ ]:
books_num_rating.rename(columns={
    "rating": "number_of_ratings"
}, inplace=True)

In [ ]:
books_num_rating.head()
books_num_rating.shape

In [ ]:
final_rating = rating_with_books.merge(books_num_rating, on="title")

In [ ]:
final_rating.head(1)

# print(final_rating.shape)

In [ ]:
final_rating = final_rating[final_rating['number_of_ratings'] >= 50]

In [ ]:
final_rating.shape

In [ ]:
final_rating.drop_duplicates(["user_id", "title"], inplace=True)

## final Rating after filtering and deleting all the duplicates ##

In [ ]:
final_rating.shape

## creating Pivot Table ##

In [ ]:
book_pivot_table = final_rating.pivot_table(columns='user_id', index="title", values= "rating")

In [ ]:
book_pivot_table

In [ ]:
book_pivot_table.shape

In [ ]:
book_pivot_table.fillna(0, inplace=True)

In [ ]:
book_pivot_table

In [ ]:
from scipy.sparse import csr_matrix  ## import this to only use those fields who have value and nullify the 0 values

In [ ]:
book_sparse = csr_matrix(book_pivot_table)

In [ ]:
book_sparse

In [ ]:
from sklearn.neighbors import NearestNeighbors

In [ ]:
book_model = NearestNeighbors(algorithm='brute')

In [ ]:
book_model.fit(book_sparse)

In [ ]:
book_pivot_table.index[38] ## this is how we see each book index

In [ ]:
distance, suggesion = book_model.kneighbors(book_pivot_table.iloc[5, :].values.reshape(1,-1), n_neighbors=5)

In [ ]:
distance

In [ ]:
suggesion

In [ ]:
for i in range(len(suggesion)):
    print(book_pivot_table.index[suggesion[i]])

In [ ]:
# book_pivot_table.index[741]
book_pivot_table.index

In [ ]:
all_book_name = book_pivot_table.index

In [ ]:
all_book_name

In [ ]:
import pickle

pickle.dump(book_model,open("Models/Book_Recc_Model.pkl", 'wb')) #creating our model || wb => write banner
pickle.dump(all_book_name,open("Models/all_book_name.pkl", 'wb'))
pickle.dump(final_rating,open("Models/final_ratings.pkl", 'wb'))
with open("Models/book_pivot.pkl", 'wb') as f:
    pickle.dump(book_pivot_table, f)

In [ ]:
def recommendedBook():
    book_name = input("Enter your book name: ").strip()
    num_of_sugg = int(input("Number of suggestions: ").strip())
    
    # Ensure book_pivot_table has a lowercased index for comparison
    lower_index = book_pivot_table.index.str.lower()
    
    # Find the book_id using the lowercased index
    try:
        book_id = np.where(lower_index == book_name.lower())[0][0]
    except IndexError:
        print("Book not found. Please try another book name.")
        return

    # Retrieve the nearest neighbors
    distance, suggestion = book_model.kneighbors(book_pivot_table.iloc[book_id, :].values.reshape(1, -1), n_neighbors=num_of_sugg)
    
    print("Recommended books:")
    for i in range(len(suggestion[0])):
        print(book_pivot_table.index[suggestion[0][i]])

# Example of how to use the function
# recommendedBook()


 ## demo of the recommended function

In [ ]:
print(recommendedBook())